In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git#egg=transformers[agents]

zsh:1: no matches found: git+https://github.com/huggingface/transformers.git#egg=transformers[agents]


In [2]:
!pip install markdownify duckduckgo-search spaces gradio-tools langchain langchain-community langchain-huggingface faiss-cpu --upgrade -q

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from transformers.agents import HfApiEngine

llm_model = "Qwen/Qwen2.5-72B-Instruct"
llm_engine = HfApiEngine(llm_model)

In [5]:
from transformers.agents import ReactCodeAgent, ReactJsonAgent, ManagedAgent
from transformers.agents.search import DuckDuckGoSearchTool, VisitWebpageTool

web_agent = ReactJsonAgent(tools=[DuckDuckGoSearchTool(), VisitWebpageTool()], llm_engine=llm_engine)

In [6]:
managed_web_agent = ManagedAgent(
    agent=web_agent,
    name="search",
    description="Runs web searches for you. Give it your query as an argument.",
)

In [7]:
import datasets

knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

In [8]:
from tqdm import tqdm
from transformers import AutoTokenizer
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]}) for doc in knowledge_base
]

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    AutoTokenizer.from_pretrained("thenlper/gte-small"),
    chunk_size=200,
    chunk_overlap=20,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

# Split docs and keep only unique ones
print("Splitting documents...")
docs_processed = []
unique_texts = {}
for doc in tqdm(source_docs):
    new_docs = text_splitter.split_documents([doc])
    for new_doc in new_docs:
        if new_doc.page_content not in unique_texts:
            unique_texts[new_doc.page_content] = True
            docs_processed.append(new_doc)

print("Embedding documents...")
embedding_model = HuggingFaceEmbeddings(model_name="thenlper/gte-small")
huggingface_doc_vector_db = FAISS.from_documents(
    documents=docs_processed,
    embedding=embedding_model,
    distance_strategy=DistanceStrategy.COSINE,
)

Splitting documents...


100%|██████████| 2647/2647 [00:27<00:00, 97.29it/s] 


Embedding documents...


In [9]:
from transformers.agents import Tool
from langchain_core.vectorstores import VectorStore


class RetrieverTool(Tool):
    name = "retriever"
    description = "Using semantic similarity, retrieves some documents from the knowledge base that have the closest embeddings to the input query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, vectordb: VectorStore, **kwargs):
        super().__init__(**kwargs)
        self.vectordb = vectordb

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.vectordb.similarity_search(
            query,
            k=7,
        )

        return "\nRetrieved documents:\n" + "".join(
            [f"===== Document {str(i)} =====\n" + doc.page_content for i, doc in enumerate(docs)]
        )

In [10]:
huggingface_doc_retriever_tool = RetrieverTool(huggingface_doc_vector_db)

In [ ]:
import os

GITHUB_ACCESS_TOKEN = os.environ.get("GITHUB_PERSONAL_TOKEN")

print(GITHUB_ACCESS_TOKEN)

In [12]:
from langchain.document_loaders import GitHubIssuesLoader

loader = GitHubIssuesLoader(repo="huggingface/peft", access_token=GITHUB_ACCESS_TOKEN, include_prs=False, state="all")
docs = loader.load()

In [13]:
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)
chunked_docs = splitter.split_documents(docs)

In [14]:
peft_issues_vector_db = FAISS.from_documents(chunked_docs, embedding=embedding_model)

In [15]:
peft_issues_retriever_tool = RetrieverTool(peft_issues_vector_db)

In [16]:
retriever_agent = ReactJsonAgent(
    tools=[huggingface_doc_retriever_tool, peft_issues_retriever_tool],
    llm_engine=llm_engine,
    max_iterations=4,
    verbose=2,
)

In [17]:
managed_retriever_agent = ManagedAgent(
    agent=retriever_agent,
    name="retriever",
    description="Retrieves documents from the knowledge base for you that are close to the input query. Give it your query as an argument. The knowledge base includes Hugging Face documentation and PEFT issues.",
)

In [18]:
from transformers import load_tool, CodeAgent

prompt_generator_tool = Tool.from_space(
    "sergiopaniego/Promptist", name="generator_tool", description="Optimizes user input into model-preferred prompts"
)

Loaded as API: https://sergiopaniego-promptist.hf.space ✔


Since `api_name` was not defined, it was automatically set to the first avilable API: `/predict`.


In [19]:
image_generation_tool = load_tool("m-ric/text-to-image", cache=False)
image_generation_agent = CodeAgent(tools=[prompt_generator_tool, image_generation_tool], llm_engine=llm_engine)

You're loading a tool from the Hub from None. Please make sure this is a source that you trust as the code within that tool will be executed on your machine. Always verify the code of the tools that you load. We recommend specifying a `revision` to ensure you're loading the code that you have checked.
TextToImageTool implements a different description in its configuration and class. Using the tool configuration description.


In [20]:
managed_image_generation_agent = ManagedAgent(
    agent=image_generation_agent,
    name="image_generation",
    description="Generates images from text prompts. Give it your prompt as an argument.",
    additional_prompting="\n\nYour final answer MUST BE only the generated image location.",
)

In [21]:
manager_agent = ReactCodeAgent(
    tools=[],
    llm_engine=llm_engine,
    managed_agents=[managed_web_agent, managed_retriever_agent, managed_image_generation_agent],
    additional_authorized_imports=["time", "datetime", "PIL"],
)

In [23]:
result = manager_agent.run(
    "Improve this prompt, then generate an image of it.", prompt="A guy wearing a suit and riding a horse."
)

======== New task ========
Improve this prompt, then generate an image of it.
You have been provided with these initial arguments: {'prompt': 'A guy wearing a suit and riding a horse.'}.
=== Agent thoughts:
Thought: improve task be_involved in this generation, so the provided argument 'prompt is: 'A guy wearing a suit and riding a horse.' I will generate an image of this prompt using the `image_generation` function.
>>> Agent is executing the code below:
image = image_generation(prompt="A guy wearing a suit and riding a horse")
final_answer(image)
====
Code execution failed due to the following error:
ManagedAgent.__call__() missing 1 required positional argument: 'request'
Traceback (most recent call last):
  File "/home/sercan/anaconda3/envs/general/lib/python3.13/site-packages/transformers/agents/agents.py", line 1196, in step
    result = self.python_evaluator(
        code_action,
    ...<3 lines>...
        authorized_imports=self.authorized_imports,
    )
  File "/home/sercan/an